<a href="https://colab.research.google.com/github/Geeteshkamble/AI/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from unsloth import FastModel
import torch
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template, standardize_data_formats, train_on_responses_only
from transformers import TextStreamer
import os
import sys
import re
import requests

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-30 06:37:30 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 04-30 06:37:30 [__init__.py:239] Automatically detected platform cuda.


In [3]:
# Available models for training
fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
]

In [4]:
def setup_colab():
    """Setup Colab environment with required dependencies."""
    if "COLAB_" in "".join(os.environ.keys()):
        print("Setting up Colab environment...")
        # Install dependencies
        !pip install --no-deps unsloth vllm
        !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
        !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

        # Handle vLLM requirements
        f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
        with open("vllm_requirements.txt", "wb") as file:
            file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
        !pip install -r vllm_requirements.txt

        # Clear some modules to avoid conflicts
        modules = list(sys.modules.keys())
        for x in modules:
            if "PIL" in x or "google" in x:
                sys.modules.pop(x)

        print("Colab environment setup complete!")

In [5]:
def load_and_prepare_dataset(csv_path):
    """Load and prepare the negative words dataset."""
    try:
        # Read the CSV file with explicit column names
        df = pd.read_csv(csv_path, encoding='utf-8', names=['Child', 'Robot'])

        # Print first few rows for debugging
        print("\nFirst few rows of the dataset:")
        print(df.head())

        # Create conversations format
        conversations = []
        for _, row in df.iterrows():
            # Skip empty rows or header row
            if pd.isna(row['Child']) or pd.isna(row['Robot']) or row['Child'] == 'Child':
                continue

            # Clean and validate the text
            child_text = str(row['Child']).strip()
            robot_text = str(row['Robot']).strip()

            if child_text and robot_text:  # Only add if both texts are non-empty
                conversation = [
                    {"role": "user", "content": [{"type": "text", "text": child_text}]},
                    {"role": "assistant", "content": [{"type": "text", "text": robot_text}]}
                ]
                conversations.append({"conversations": conversation})

        if not conversations:
            raise ValueError("No valid conversations found in the dataset")

        # Convert to HuggingFace dataset
        dataset = Dataset.from_list(conversations)
        print(f"\nSuccessfully created {len(conversations)} conversation pairs")
        return dataset

    except pd.errors.EmptyDataError:
        print("Error: The CSV file is empty")
        raise
    except pd.errors.ParserError:
        print("Error: The CSV file is not properly formatted")
        raise
    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        print("\nPlease ensure your CSV file:")
        print("1. Has two columns (negative statement and positive response)")
        print("2. Is properly formatted with commas as separators")
        print("3. Contains valid text in both columns")
        raise


In [6]:
def format_dataset(dataset, tokenizer):
    """Format the dataset using the chat template."""
    def formatting_prompts_func(examples):
        convos = examples["conversations"]
        texts = []
        for convo in convos:
            # Format each conversation into a single text string
            formatted_text = ""
            for message in convo:
                role = message["role"]
                content = message["content"][0]["text"]
                if role == "user":
                    formatted_text += f"<start_of_turn>user\n{content}<end_of_turn>\n"
                else:  # assistant
                    formatted_text += f"<start_of_turn>model\n{content}<end_of_turn>\n"
            texts.append(formatted_text)
        return {"text": texts}

    return dataset.map(formatting_prompts_func, batched=True)

In [7]:
def train_model(dataset, model_name="unsloth/gemma-3-4b-it-unsloth-bnb-4bit", max_steps=60):
    """Train the Gemma-3 model on the dataset."""
    # Initialize model and tokenizer
    model, tokenizer = FastModel.from_pretrained(
        model_name=model_name,
        max_seq_length=2048,  # Choose any for long context!
        load_in_4bit=True,    # 4 bit quantization to reduce memory
        load_in_8bit=False,   # A bit more accurate, uses 2x memory
        full_finetuning=False, # We have full finetuning now!
    )

    # Get PEFT model
    model = FastModel.get_peft_model(
        model,
        finetune_vision_layers=False,  # Turn off for just text!
        finetune_language_layers=True,  # Should leave on!
        finetune_attention_modules=True,  # Attention good for GRPO
        finetune_mlp_modules=True,  # Should leave on always!
        r=8,           # Larger = higher accuracy, but might overfit
        lora_alpha=8,  # Recommended alpha == r at least
        lora_dropout=0,
        bias="none",
        random_state=3407,
    )

    # Format dataset
    print("\nFormatting dataset for training...")
    dataset = format_dataset(dataset, tokenizer)

    # Initialize trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        eval_dataset=None,
        args=SFTConfig(
            dataset_text_field="text",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,  # Use GA to mimic batch size!
            warmup_steps=5,
            max_steps=max_steps,
            learning_rate=2e-4,  # Reduce to 2e-5 for long training runs
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            report_to="none",  # Use this for WandB etc
            dataset_num_proc=2,
        ),
    )

    # Train the model
    print("\nStarting training...")
    trainer_stats = trainer.train()

    return model, tokenizer, trainer_stats


In [23]:
def test_model(model, tokenizer, test_inputs):
    """Test the trained model with sample inputs."""
    results = []
    for input_text in test_inputs:
        messages = [{
            "role": "user",
            "content": [{"type": "text", "text": input_text}]
        }]

        text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
        )

        outputs = model.generate(
            **tokenizer([text], return_tensors="pt").to("cuda"),
            max_new_tokens=200,  # Increase for longer outputs!
            temperature=1.0,    # Recommended Gemma-3 settings!
            top_p=0.95,         # Recommended Gemma-3 settings!
            top_k=64,           # Recommended Gemma-3 settings!
        )

        response = tokenizer.batch_decode(outputs)[0]
        results.append({"input": input_text, "response": response})

    return results

In [ ]:
def main():
    # Setup Colab environment if running in Colab
    setup_colab()

    # Load and prepare dataset
    print("Loading and preparing dataset...")
    try:
        dataset = load_and_prepare_dataset("Negative words.csv")
    except Exception as e:
        print(f"\nFailed to load dataset: {str(e)}")
        print("\nPlease check your CSV file and try again.")
        return

    # Train model
    print("\nStarting model training...")
    try:
        model, tokenizer, trainer_stats = train_model(dataset)
        print(f"\nTraining completed in {trainer_stats.metrics['train_runtime']} seconds")
        print(f"Training completed in {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
    except Exception as e:
        print(f"\nError during training: {str(e)}")
        return

    # Show memory stats
    try:
        gpu_stats = torch.cuda.get_device_properties(0)
        used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
        max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
        print(f"\nGPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
        print(f"Peak reserved memory = {used_memory} GB.")
        print(f"Peak reserved memory % of max memory = {round(used_memory / max_memory * 100, 3)} %.")
    except Exception as e:
        print(f"\nError getting GPU stats: {str(e)}")

    # Save the model
    print("\nSaving model...")
    try:
        model.save_pretrained("gemma-3-negative-words")
        tokenizer.save_pretrained("gemma-3-negative-words")
        print("Model saved successfully")
    except Exception as e:
        print(f"\nError saving model: {str(e)}")
        return

    # Test cases
    test_inputs = [
        # "I don't want to do my homework!",
        # "I don't like vegetables!",
        # "I don't want to go to school today!",
        # "I don't want to share my toys!",
        # "I don't want to go to bed!",
        # "I don't want to clean my room!",
        # "I don't want to help with dinner!",
        # "I don't want to play outside!",
        # "I don't want to take a bath!",
        # "I don't want to exercise!"
        "i peed in my pants while sleeping"
    ]

    print("\nTesting model with sample inputs:")
    try:
        results = test_model(model, tokenizer, test_inputs)

        for result in results:
            print(f"\nInput: {result['input']}")
            print(f"Response: {result['response']}")
    except Exception as e:
        print(f"\nError during testing: {str(e)}")
        return

    # Save to float16 for deployment
    print("\nSaving model in float16 format for deployment...")
    try:
        model.save_pretrained_merged("gemma-3-negative-words-float16", tokenizer)
        print("Float16 model saved successfully")
    except Exception as e:
        print(f"\nError saving float16 model: {str(e)}")

    # Save to GGUF format
    print("\nSaving model in GGUF format...")
    try:
        model.save_pretrained_gguf(
            "gemma-3-negative-words-gguf",
            quantization_type="Q8_0",  # For now only Q8_0, BF16, F16 supported
        )
        print("GGUF model saved successfully")
    except Exception as e:
        print(f"\nError saving GGUF model: {str(e)}")

if __name__ == "__main__":
    main()